# 供应网络设计 1

## 目标和准备条件

尝试这个 Jupyter Notebook 建模示例，学习如何解决一个经典的供应网络设计问题，该问题涉及找到网络中的最小成本流。我们将向您展示如何 – 在给定一组工厂、仓库和客户的情况下 – 使用数学优化来确定满足客户需求的最佳方式，同时最小化运输成本。

这个模型是 H. Paul Williams 所著《数学规划中的模型构建》第五版第 273-275 页和 330-332 页中的示例 19。

这是一个初级难度的示例；我们假设您了解 Python，并对 Gurobi Python API 和构建数学优化模型有一定了解。

**下载存储库** <br />
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的存储库。

---
## 问题描述

在这个问题中，我们有六个终端客户，每个客户对产品都有已知需求。客户需求可以从四个仓库或直接从两个工厂满足。每个仓库可以支持通过它的最大产品数量，每个工厂可以生产最大数量的产品。在运输产品时，从工厂到仓库、从仓库到客户、或从工厂直接到客户都有已知的成本。

我们的供应网络有两个工厂，分别位于利物浦和布莱顿，生产产品。每个工厂都有最大生产能力：

| 工厂 | 供应量（吨） |
| --- | --- |
| 利物浦 | 150,000 |
| 布莱顿 | 200,000 |

产品可以从工厂运送到四个仓库。每个仓库都有最大吞吐量。仓库不生产或消耗产品；它们只是将产品传递给客户。

| 仓库 | 吞吐量（吨） |
| --- | --- |
| 纽卡斯尔 | 70,000 |
| 伯明翰 | 50,000 |
| 伦敦 | 100,000 |
| 埃克塞特 | 40,000 |

我们的网络有六个客户，每个客户都有特定需求。

| 客户 | 需求量（吨） |
| --- | --- |
| C1 | 50,000 |
| C2 | 10,000 |
| C3 | 40,000 |
| C4 | 35,000 |
| C5 | 60,000 |
| C6 | 20,000 |

运输成本在下表中给出（以美元/吨为单位）。列是源城市，行是目的地城市。例如，从利物浦到伦敦运输产品的成本为每吨1美元。表中的"-"表示该组合不可能，例如不可能从布莱顿工厂运输到纽卡斯尔仓库。

| 目的地 | 利物浦 | 布莱顿 | 纽卡斯尔 | 伯明翰 | 伦敦 | 埃克塞特 |
| --- | --- | --- | --- | --- | --- | --- |
| 仓库 |
| 纽卡斯尔 | 0.5 | - |
| 伯明翰 | 0.5 | 0.3 |
| 伦敦 | 1.0 | 0.5 |
| 埃克塞特 | 0.2 | 0.2 |
| 客户 |
| C1 | 1.0 | 2.0 | - | 1.0 | - | - |
| C2 | - | - | 1.5 | 0.5 | 1.5 | - |
| C3 | 1.5 | - | 0.5 | 0.5 | 2.0 | 0.2 |
| C4 | 2.0 | - | 1.5 | 1.0 | - | 1.5 |
| C5 | - | - | - | 0.5 | 0.5 | 0.5 |
| C6 | 1.0 | - | 1.0 | - | 1.5 | 1.5 |

需要解决的问题是如何在最小化运输成本的同时满足终端客户的需求。

---
## 模型公式

### 集合和索引

$f \in \text{工厂}=\{\text{利物浦}, \text{布莱顿}\}$

$d \in \text{仓库}=\{\text{纽卡斯尔}, \text{伯明翰}, \text{伦敦}, \text{埃克塞特}\}$

$c \in \text{客户}=\{\text{C1}, \text{C2}, \text{C3}, \text{C4}, \text{C5}, \text{C6}\}$

$\text{城市} = \text{工厂} \cup \text{仓库} \cup \text{客户}$

### 参数

$\text{cost}_{s,t} \in \mathbb{R}^+$: 从源点 $s$ 到目的地 $t$ 运输一吨的成本。

$\text{supply}_f \in \mathbb{R}^+$: 工厂 $f$ 的最大可能供应量（吨）。

$\text{through}_d \in \mathbb{R}^+$: 仓库 $d$ 的最大可能流通量（吨）。

$\text{demand}_c \in \mathbb{R}^+$: 客户 $c$ 的产品需求量（吨）。

### 决策变量

$\text{flow}_{s,t} \in \mathbb{N}^+$: 从源点 $s$ 到目的地 $t$ 运输的产品数量（吨）。


### 目标函数

- **成本**: 最小化总运输成本。

\begin{equation}
\text{最小化} \quad Z = \sum_{(s,t) \in \text{城市} \times \text{城市}}{\text{cost}_{s,t}*\text{flow}_{s,t}}
\end{equation}

### 约束条件

- **工厂产出**: 工厂的产品流出量必须遵守最大产能。

\begin{equation}
\sum_{t \in \text{城市}}{\text{flow}_{f,t}} \leq \text{supply}_{f} \quad \forall f \in \text{工厂}
\end{equation}

- **客户需求**: 产品流量必须满足客户需求。

\begin{equation}
\sum_{s \in \text{城市}}{\text{flow}_{s,c}} = \text{demand}_{c} \quad \forall c \in \text{客户}
\end{equation}

- **仓库流量**: 进入仓库的流量等于从仓库流出的流量。

\begin{equation}
\sum_{s \in \text{城市}}{\text{flow}_{s,d}} = 
\sum_{t \in \text{城市}}{\text{flow}_{d,t}}
\quad \forall d \in \text{仓库}
\end{equation}

- **仓库容量**: 进入仓库的流量必须遵守仓库容量。

\begin{equation}
\sum_{s \in \text{城市}}{\text{flow}_{s,d}} \leq \text{through}_{d}
\quad \forall d \in \text{仓库}
\end{equation}

---
## Python 实现

我们导入 Gurobi Python 模块和其他 Python 库。

In [ ]:
# %pip install gurobipy

In [ ]:
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# 使用 Python 3.11 和 Gurobi 11.0 测试通过

## 输入数据
我们定义模型的所有输入数据。

In [ ]:
# 创建字典来存储工厂供应限制、仓库吞吐限制和客户需求

supply = dict({'Liverpool': 150000,
               'Brighton': 200000})

through = dict({'Newcastle': 70000,
                'Birmingham': 50000,
                'London': 100000,
                'Exeter': 40000})

demand = dict({'C1': 50000,
               'C2': 10000,
               'C3': 40000,
               'C4': 35000,
               'C5': 60000,
               'C6': 20000})

# 创建字典来存储运输成本

arcs, cost = gp.multidict({
    ('Liverpool', 'Newcastle'): 0.5,
    ('Liverpool', 'Birmingham'): 0.5,
    ('Liverpool', 'London'): 1.0,
    ('Liverpool', 'Exeter'): 0.2,
    ('Liverpool', 'C1'): 1.0,
    ('Liverpool', 'C3'): 1.5,
    ('Liverpool', 'C4'): 2.0,
    ('Liverpool', 'C6'): 1.0,
    ('Brighton', 'Birmingham'): 0.3,
    ('Brighton', 'London'): 0.5,
    ('Brighton', 'Exeter'): 0.2,
    ('Brighton', 'C1'): 2.0,
    ('Newcastle', 'C2'): 1.5,
    ('Newcastle', 'C3'): 0.5,
    ('Newcastle', 'C5'): 1.5,
    ('Newcastle', 'C6'): 1.0,
    ('Birmingham', 'C1'): 1.0,
    ('Birmingham', 'C2'): 0.5,
    ('Birmingham', 'C3'): 0.5,
    ('Birmingham', 'C4'): 1.0,
    ('Birmingham', 'C5'): 0.5,
    ('London', 'C2'): 1.5,
    ('London', 'C3'): 2.0,
    ('London', 'C5'): 0.5,
    ('London', 'C6'): 1.5,
    ('Exeter', 'C3'): 0.2,
    ('Exeter', 'C4'): 1.5,
    ('Exeter', 'C5'): 0.5,
    ('Exeter', 'C6'): 1.5
})

## 模型部署

我们创建一个模型和变量。这些变量简单地捕获在源点和目的地之间允许路径上流动的产品数量。优化目标系数在此处提供（在 $\text{cost}$ 中），因此我们之后不需要提供优化目标。

In [3]:
model = gp.Model('SupplyNetworkDesign')
flow = model.addVars(arcs, obj=cost, name="flow")

Using license file c:\gurobi\gurobi.lic


我们的第一个约束要求离开工厂的总流量不超过该工厂的供应能力。

In [ ]:
# 生产能力限制

factories = supply.keys()
factory_flow = model.addConstrs((gp.quicksum(flow.select(factory, '*')) <= supply[factory]
                                 for factory in factories), name="factory")

我们的下一个约束要求进入客户的总流量等于该客户的需求。

In [ ]:
# 客户需求

customers = demand.keys()
customer_flow = model.addConstrs((gp.quicksum(flow.select('*', customer)) == demand[customer]
                                  for customer in customers), name="customer")

我们的最后约束与仓库有关。第一个约束要求进入仓库的产品总量必须等于离开的总量。

In [ ]:
# 仓库流量平衡

depots = through.keys()
depot_flow = model.addConstrs((gp.quicksum(flow.select(depot, '*')) == gp.quicksum(flow.select('*', depot))
                               for depot in depots), name="depot")

第二组约束限制通过仓库的产品量最多等于该仓库的吞吐量。

In [ ]:
# 仓库吞吐量

depot_capacity = model.addConstrs((gp.quicksum(flow.select('*', depot)) <= through[depot]
                                   for depot in depots), name="depot_capacity")

现在我们优化模型

In [8]:
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 16 rows, 29 columns and 65 nonzeros
Model fingerprint: 0x3607c855
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve removed 1 rows and 0 columns
Presolve time: 0.03s
Presolved: 15 rows, 29 columns, 64 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4800000e+05   1.812500e+04   0.000000e+00      0s
       7    1.9850000e+05   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.03 seconds
Optimal objective  1.985000000e+05


---
## 分析

所有客户的产品需求都可以以总成本 $\$198,500$ 得到满足。最优方案如下。

In [9]:
product_flow = pd.DataFrame(
    [{"From": arc[0], "To": arc[1], "Flow": flow[arc].x} for arc in arcs if flow[arc].x > 1e-6]
)
product_flow.index=[''] * len(product_flow)
product_flow

,From,To,Flow
,Liverpool,C1,50000.0
,Liverpool,C6,20000.0
,Brighton,Birmingham,50000.0
,Brighton,London,55000.0
,Brighton,Exeter,40000.0
,Birmingham,C2,10000.0
,Birmingham,C4,35000.0
,Birmingham,C5,5000.0
,London,C5,55000.0
,Exeter,C3,40000.0


---
## 参考文献

H. Paul Williams，《数学规划中的模型构建》，第五版。

版权所有 © 2020 Gurobi Optimization, LLC